In [21]:
import requests
import selenium
from selenium.webdriver.common.by import By
from selenium import webdriver
from bs4 import BeautifulSoup as bs

from tqdm import tqdm

import time
from datetime import datetime

import pandas as pd

In [22]:
def get_reviewers(res):
    result = pd.DataFrame()
    
    for reviewer in tqdm(res.json()['topReviewerList']):
        cols = reviewer.keys()

        tmp = dict()
        for col in cols:
            if col in ['gdas','photoList'] : continue
                
            if reviewer[col]:
                tmp[col] = reviewer[col]

        result = pd.concat([result, pd.DataFrame(tmp, index=[0])])

    return result.reset_index(drop=True)


In [23]:
def get_topReviewer():
    url = 'https://www.oliveyoung.co.kr/store/mypage/getTopReviewerListJSON.do'
    
    page = 1
    params = {
        'pageIdx' : page
    }
    
    result = pd.DataFrame()
    before_members = []
    
    cnt = 1
    while True:
        time.sleep(0.3)
        
        res = requests.get(url, params=params)
            
        reviewer = get_reviewers(res)
                
        result = pd.concat([result, reviewer])
        
        if len(reviewer['mbrNm'].values) < 10:
            break
            
        page +=1
        params['pageIdx'] = page
        
            
    return result.reset_index(drop=True)

In [24]:
reviewer = get_topReviewer()

  0%|                                                    | 0/10 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [474]:
reviewer.to_csv('topReviewers.csv', index = False)

In [25]:
reviewer = pd.read_csv('topReviewers.csv')

In [83]:
def get_reviews(members, end_date):
    url = 'https://www.oliveyoung.co.kr/store/mypage/getReviewerGdasListJSON.do'

    params = {
        'gdasSort': '03'
    }
    last_content = ''

    reviews = pd.DataFrame()

    for mem_no in (members['mbrNo'].values):
        params['mbrNo'] = mem_no
#         print(mem_no)
        member_review = pd.DataFrame()

        page = 1
        while True:
            params['pageIdx'] = page
            res = requests.get(url, params=params)

            reviews = pd.concat([reviews, member_reviews(res)])

            writed_at = reviews['dispRegDate'].iloc[-1].split('.')
            writed_at = list(map(int, writed_at))
            writed_at = datetime(writed_at[0],writed_at[1],writed_at[2])
            


            if (end_date- writed_at).days > 60: # 최근 2달 60일로 계산
#                 print('Over 60 day')
                break
            
            if last_content == reviews['gdasCont'].iloc[-1]:
#                 print('Under 60 day')
                break
            
            last_content = reviews['gdasCont'].iloc[-1]

            page+=1
            
    return reviews



In [84]:
def member_reviews(res):
    
    result = pd.DataFrame()
    
    for reviewer in res.json()['allReviewList']:
        cols = reviewer.keys()

        tmp = dict()
        for col in cols:
            
            if col == 'photoList':
                if reviewer[col]:
                    tmp[col] = len(reviewer[col])
                else:
                    tmp[col] = 0
                    
            elif reviewer[col]:
                tmp[col] = str(reviewer[col])
            
        result = pd.concat([result, pd.DataFrame(tmp,index = [0])])

    return result.reset_index(drop=True)

In [85]:
end_date = datetime(2023,4,21)

slice_cnt = 30

reviews = pd.DataFrame()
from tqdm import tqdm
for cnt in tqdm(range(int(len(reviewer)/30)+1)):
    
    tmp = get_reviews(reviewer[cnt*slice_cnt:(cnt+1)*slice_cnt], end_date)
    reviews = pd.concat([reviews, tmp])

100%|███████████████████████████████████████████| 68/68 [43:54<00:00, 38.74s/it]


In [90]:
reviews = reviews.reset_index(drop=True)

In [91]:
reviews.to_csv('./topReviews.csv', index=False)

In [238]:
drop_cols = ['itemNo', 'pageIdx','lgcGoodsNo', 'dbLocaleLanguage', 'itemNm', 'profileOpenYn','colDataCdArr', 'rownum', 'prGoodsSumrInfo', 'profileImg', 'addInfoNm', 'skinTypeCd','skinToneCd','skinTroubleCd', 'gdasPrhbCatCnt']

In [241]:
test = reviews.drop(columns=drop_cols, axis=1)
test['gdasSeq'] = test['gdasSeq'].astype('int32')
test['gdasScrVal'] = test['gdasScrVal'].astype('int8')
test['gdasSctCd'] = test['gdasSctCd'].astype('int8')
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121949 entries, 0 to 121948
Data columns (total 17 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   gdasSeq             121949 non-null  int32 
 1   goodsNo             121949 non-null  object
 2   gdasScrVal          121949 non-null  int8  
 3   gdasCont            121945 non-null  object
 4   mbrNo               121949 non-null  object
 5   gdasSctCd           121949 non-null  int8  
 6   ordDate             115041 non-null  object
 7   dispRegDate         121949 non-null  object
 8   myRecommYn          121949 non-null  object
 9   photoList           121949 non-null  int64 
 10  addInfoOpenYn       121949 non-null  object
 11  showFilter          121949 non-null  object
 12  firstGdasYn         121943 non-null  object
 13  renewUsed1mmGdasYn  121890 non-null  object
 14  recommCnt           40939 non-null   object
 15  hshTag              9942 non-null    object
 16  us